In [107]:
import asyncio
import json
from datetime import datetime
import requests
import hashlib
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

import requests
import web3
import pandas as pd

import sys
sys.path.append("..")

import chainklik.libs.common.utils as utils
import chainklik.libs.aave.api as aave_api
import chainklik.libs.common.payload as payload
import chainklik.libs.common.etherscan as etherscan
import eth_abi
import chainklik.config.config as cfg

import warnings
warnings.filterwarnings('ignore')

In [108]:
es = Elasticsearch(
    "https://localhost:9200",
    http_auth=('elastic', 'y=fUp=8ucKL18I5K=1Am'),
    verify_certs=False
)

In [109]:
def es_create_index(index_name):
    try:
        es.indices.create(index=index_name)
    except Exception as e:
        print(e)

In [344]:
index_name = 'abi'
es_create_index(index_name)
es.indices.delete(index=index_name, ignore=[400, 404])

BadRequestError(400, 'resource_already_exists_exception', 'index [abi/s8jiox6UQrOLp0c7hL9y9A] already exists')


ObjectApiResponse({'acknowledged': True})

In [345]:
es.indices.create(
    index=index_name,
    body={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0,

            "analysis": {
                "analyzer": {
                    "lc_analyzer": {
                        "tokenizer": "lowercase",
                    },
                    "addr_analyzer" : {
                        "tokenizer": "edge_ngram_tokenizer"
                    }
                },
                "tokenizer": {
                    "edge_ngram_tokenizer": {
                        "type": "edge_ngram",
                        "min_gram": 8,
                        "max_gram": 42,
                        "token_chars": ["letter", "digit"]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "name": {
                    "type": "text",
                    "analyzer": "lc_analyzer" 
                },
                "contract": {
                    "type": "text",
                    "analyzer": "lc_analyzer"
                },
                "type": {
                    "type": "text",
                    "analyzer": "lc_analyzer"
                },
                "address": {
                    "type": "text",
                    "analyzer": "addr_analyzer"
                }
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'abi'})

In [346]:
# use aave v2 as an example
ws = web3.Web3(web3.Web3.HTTPProvider(cfg.config["http_url"]))

In [347]:
def doc_id(contract_addr, abi):
    return hashlib.md5('{}/{}'.format(contract_addr,str(abi)).encode()).hexdigest()

In [348]:
def add_abi(contract_addr, impl_addr, contract_name, author = "rshi"):
    # if contract addr is different from impl addr
    pool_contract = etherscan.get_contract(contract_addr,impl_addr)
    # save contract abi
    doc = {
        'author': 'rshi',
        'timestamp': datetime.now(),
        'id' : doc_id(contract_addr, pool_contract.abi),
        'address' : contract_addr,
        'contract' : contract_name,
        'type' : "contract",
        'name' : contract_name,
        'abi': pool_contract.abi
    }
    resp = es.index(index='abi', id=doc['id'], document=doc)

    # save contract function and event abi
    for abi in pool_contract.abi:
        if abi["type"] != "constructor":
            doc = {
                'author': 'rshi',
                'timestamp': datetime.now(),
                'id' : doc_id(contract_addr, abi),
                'address' : contract_addr,
                'contract' : contract_name,
                'type' : abi["type"],
                'name' : abi["name"],
                'abi': abi
            }
            resp = es.index(index='abi', id=doc['id'], document=doc)

In [342]:
contract_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"
impl_addr = contract_addr
contract_name = "Curve.fi"
add_abi(contract_addr, impl_addr, contract_name)

In [357]:
contract_addr = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
impl_addr = "0x085E34722e04567Df9E6d2c32e82fd74f3342e79"
contract_name = "Aave v2"
add_abi(contract_addr, impl_addr, contract_name)

In [356]:
s = Search(using=es, index=index_name)
# keywords = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
# keyword_queries = []
    
# for keyword in keywords:
#     keyword_queries.append(Q("multi_match", query=keyword, fields=["_all"]))

# combined_query = Q('bool', should=keyword_queries)
# s = s.query(combined_query)

# query = Q('bool',
#     must=[
#         Q('match', address="0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"),
#         Q('match', type='function'), 
#         Q('match', name='get_dy')
#     ]
# )

page_size = 50  # Number of results per page
page_number = 1  # Page number (starting from 1)

# Calculate the starting point (from) based on page size and page number
from_index = (page_number - 1) * page_size

# Set the pagination parameters in the search query
s = s[from_index:from_index + page_size]

query = Q('bool',
    should=[
        Q('multi_match', query="0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9", fields=["address"]),
        Q('multi_match', query="Curve", fields=["name","contract","type"])
    ]
)

s = s.query(query)

response = s.execute()

response.to_dict()["hits"]["hits"][0]

{'_index': 'abi',
 '_id': '58aece5b6572a987bde6a09ac8ab5f39',
 '_score': 3.5964208,
 '_source': {'author': 'rshi',
  'timestamp': '2023-12-03T13:20:03.202761',
  'id': '58aece5b6572a987bde6a09ac8ab5f39',
  'address': '0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7',
  'contract': 'Curve 3pool',
  'type': 'contract',
  'name': 'Curve 3pool',
  'abi': [{'name': 'TokenExchange',
    'inputs': [{'type': 'address', 'name': 'buyer', 'indexed': True},
     {'type': 'int128', 'name': 'sold_id', 'indexed': False},
     {'type': 'uint256', 'name': 'tokens_sold', 'indexed': False},
     {'type': 'int128', 'name': 'bought_id', 'indexed': False},
     {'type': 'uint256', 'name': 'tokens_bought', 'indexed': False}],
    'anonymous': False,
    'type': 'event'},
   {'name': 'AddLiquidity',
    'inputs': [{'type': 'address', 'name': 'provider', 'indexed': True},
     {'type': 'uint256[3]', 'name': 'token_amounts', 'indexed': False},
     {'type': 'uint256[3]', 'name': 'fees', 'indexed': False},
     {'typ

In [336]:
from elasticsearch.client import IndicesClient

indices_client = IndicesClient(es)

# Use the _analyze endpoint to test the custom analyzer
analysis_results = indices_client.analyze(
    index=index_name,
    body={
        "analyzer": "addr_analyzer",
        "text": "0xabfdasfas"
    }
)

In [337]:
analysis_results

ObjectApiResponse({'tokens': [{'token': '0xabfdas', 'start_offset': 0, 'end_offset': 8, 'type': 'word', 'position': 0}, {'token': '0xabfdasf', 'start_offset': 0, 'end_offset': 9, 'type': 'word', 'position': 1}, {'token': '0xabfdasfa', 'start_offset': 0, 'end_offset': 10, 'type': 'word', 'position': 2}, {'token': '0xabfdasfas', 'start_offset': 0, 'end_offset': 11, 'type': 'word', 'position': 3}]})